# funcX Tutorial

funcX is a Function-as-a-Service (FaaS) platform for science that enables you to convert almost any computing resource into a high-performance function serving device. Deploying a funcX endpoint will integrate your resource into the function serving fabric, allowing you to dynamically send, monitor, and receive results from function invocations. funcX is built on top of Parsl, allowing you to connect your endpoint to large compute resources via traditional batch queues, where funcX will dynamically provision, use, and release resources on-demand to fulfill function requests.

Here we provide an example of using funcX to register a function and run it on a publicly available tutorial endpoint.

We start by creating a funcX client to interact with the service.

In [1]:
from funcx.sdk.client import FuncXClient

fxc = FuncXClient()

Here we define the tutorial endpoint to be used in this demonstration. Because the tutorial endpoint is Kubernetes-based, we select a simple python3.6 container that will be used during execution.

In [2]:
def funcx_sum(items):
    return sum(items)

In [3]:
func_uuid = fxc.register_function(funcx_sum,
                                  description="A sum function")
print(func_uuid)

4cb9a107-7ad9-41ca-bfb8-2b91b9c2983e


In [4]:
payload = [1, 2, 3, 4, 66]

endpoint_uuid = '840b214f-ea5c-4d0c-b2b8-ea591634065b'

res = fxc.run(payload, endpoint_id=endpoint_uuid, function_id=func_uuid)
print(res)

949b10ba-85ce-4203-a270-0a42ed3edfec


In [5]:
fxc.get_result(res)

76